I have tried using both TF-IDF and Count Vectorizer  <br>
* Logistic regression
* Naive bayes
* Xgboost
* ANN
* CNN

Models are getting confused for rating 3, few of them are predicted as 4 or 2

I also wanted to try GloVe, but I was getting memory issues, I kept the code, later I will try.

Kindly upvote if you like this notebook.<br>
Any issues or mistake kindly let me know in comments, happy to correct.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from nltk import word_tokenize
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import math

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import string
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
data = pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')

In [ ]:
sns.countplot(data=data,x='Rating')

* Data is highly imbalanced.

**Cleaning of data**

In [ ]:
def  clean_text(text):
    text = text.str.lower()
    text = text.apply(lambda T: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", T))          
    return text

data['Review']= clean_text(data['Review'])
data['Review'] = data['Review'].str.replace('#','')
data['Review'] = data['Review'].str.lower()

punk_Remove = string.punctuation

def del_punk(text):
    return text.translate(str.maketrans('','',punk_Remove))

data['Review'] = data['Review'].apply(lambda T: del_punk(T))

In [ ]:
stop_word = set(stopwords.words('english'))
def del_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_word])

data['Review'] = data['Review'].apply(lambda T: del_stopwords(T))
lemmatizer = WordNetLemmatizer()
wordnet_m = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}


def lemmat_pos_word(text):
    
    pos_tagger_text = nltk.pos_tag(text.split())
    
    return " ".join([lemmatizer.lemmatize(word, wordnet_m.get(pos[0],wordnet.NOUN)) for word,pos in pos_tagger_text])



data['Review'] = data['Review'].apply(lambda T : lemmat_pos_word(T))

In [ ]:
data['totalwords'] = data['Review'].str.count(' ') + 1

In [ ]:
plt.figure(figsize=(16,16))
for i, rating in enumerate([1,2,3,4,5]):
    plt.subplot(4,4,i+1)
    plt.title("Rating "+str(rating))
    sns.distplot(data[data['Rating']==rating]['totalwords'],kde=False)
    plt.tight_layout()

Usually these comments are power law, I will check for it later 

In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [ ]:
data.head()

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.Rating.values)

In [ ]:
data.columns

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(data.Review.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
xtrain, xvalid, ytrain, yvalid = train_test_split(pd.DataFrame(xtrain)[0], ytrain, 
                                                  stratify=ytrain, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

**Using td Idf**

In [ ]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

tfv.fit(list(xtrain) + list(xvalid)+ list(xtest))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)
xtest_tfv = tfv.transform(xtest)

In [ ]:
LR_tfv_clf = LogisticRegression()
LR_tfv_clf.fit(xtrain_tfv, ytrain)
predictions = LR_tfv_clf.predict_proba(xvalid_tfv)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions));
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

**Using count Vectorizer**

In [ ]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 3), stop_words = 'english')

ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [ ]:
LR_ctv_clf = LogisticRegression()
LR_ctv_clf.fit(xtrain_ctv, ytrain)
predictions = LR_ctv_clf.predict_proba(xvalid_ctv)

print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
NMB_ctv_clf = MultinomialNB()
NMB_ctv_clf.fit(xtrain_ctv, ytrain)
predictions = NMB_ctv_clf.predict_proba(xvalid_ctv)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
NMB_tfv_clf = MultinomialNB()
NMB_tfv_clf.fit(xtrain_tfv, ytrain)
predictions = NMB_tfv_clf.predict_proba(xvalid_tfv)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
XGB_ctv_clf = xgb.XGBClassifier()
XGB_ctv_clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = XGB_ctv_clf.predict_proba(xvalid_ctv.tocsc())

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
XGB_tfv_clf = xgb.XGBClassifier()
XGB_tfv_clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = XGB_tfv_clf.predict_proba(xvalid_tfv.tocsc())

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
0

Logistic Regression gave a better results, Lets tune it.<br>


In [ ]:
C = [10**x for x in range(-5,5)]
tuned_parameters = [{'C': C}]
model = GridSearchCV(LogisticRegression(class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=3,return_train_score = True)
model.fit(xtrain_tfv, ytrain);

In [ ]:
print(model.best_estimator_)
results = pd.DataFrame.from_dict(model.cv_results_)
results = results.sort_values(['param_C'])

train_score= results['mean_train_score']
train_acc_std= results['std_train_score']
cv_score = results['mean_test_score'] 
cv_acc_std= results['std_test_score']
C =  results['param_C']

C = [math.log(x,10) for x in C]
plt.plot(C, train_score, label='Train')

plt.plot(C, cv_acc_std, label='CV')

plt.scatter(C, train_score, label='Train points')
plt.scatter(C, cv_acc_std, label='CV points')

plt.legend()
plt.xlabel("C: hyperparameter")
plt.ylabel("Acc")
plt.grid()
plt.show()

In [ ]:
LR_ctv_clf = LogisticRegression(C=1, class_weight='balanced')
LR_ctv_clf.fit(xtrain_ctv, ytrain)
predictions = LR_ctv_clf.predict_proba(xvalid_ctv)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

Lets Tune XGBoost

In [ ]:
params = {
        'n_estimators': [ 500, 1000],
        'max_depth': [4, 5, 8],
        'tree_method':['gpu_hist']
        }
model = GridSearchCV(xgb.XGBClassifier(), params, scoring = 'accuracy', cv=3,return_train_score = True)
model.fit(xtrain_tfv.tocsc(), ytrain)

print(model.best_estimator_)
results = pd.DataFrame.from_dict(model.cv_results_)
results = results.sort_values(['param_C'])

train_score= results['mean_train_score']
train_acc_std= results['std_train_score']
cv_score = results['mean_test_score'] 
cv_acc_std= results['std_test_score']
C =  results['param_C']

C = [math.log(x,10) for x in C]
plt.plot(C, train_acc, label='Train')

plt.plot(C, cv_acc, label='CV')

plt.scatter(C, train_acc, label='Train points')
plt.scatter(C, cv_acc, label='CV points')

plt.legend()
plt.grid()
plt.show()

In [ ]:
XGB_tfv_clf = xgb.XGBClassifier()
XGB_tfv_clf.fit(xtrain_tfv.tocsc(), ytrain_tfv)
predictions = XGB_tfv_clf.predict_proba(xvalid_tfv.tocsc())

print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

Trying ANN

In [ ]:
tfidf = TfidfVectorizer(binary=True)
X_train = tfidf.fit_transform(xtrain).astype('float16')
X_test = tfidf.transform(xtest).astype('float16')
X_valid = tfidf.transform(xvalid).astype('float16')

In [ ]:
lb = preprocessing.LabelEncoder()
y = lb.fit_transform(ytrain)
y_train = np_utils.to_categorical(ytrain)
y_test= np_utils.to_categorical(ytest)
y_valid = np_utils.to_categorical(yvalid)

In [ ]:
X_train.shape

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=65413, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
# model.add(Dense(216, activation='relu'))
# model.add(Dropout(0.8))
# model.add(BatchNormalization())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.8))
# model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y=y_train, batch_size=128,epochs=10, verbose=1,validation_data=(X_valid, y_valid))

In [ ]:
pred=model.predict_classes(X_test)
print(classification_report(ytest,pred))
print(confusion_matrix(ytest, pred))

Trying CNN

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
def encode_docs(tokenizer, max_length, docs):
    encoded = tokenizer.texts_to_sequences(docs)
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

In [ ]:
tokenizer = create_tokenizer(xtrain)
vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(s.split()) for s in xtrain])

In [ ]:
Xtrain = encode_docs(tokenizer, max_length, xtrain)

In [ ]:
lb = preprocessing.LabelEncoder()
y = lb.fit_transform(ytrain)
y_train = np_utils.to_categorical(ytrain)
y_test= np_utils.to_categorical(ytest)
y_valid = np_utils.to_categorical(yvalid)

In [ ]:
Xvalid = encode_docs(tokenizer, max_length, xvalid)
Xtest = encode_docs(tokenizer, max_length, xtest)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_length))
model.add(Conv1D(128, 8, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(64, 8, activation='relu'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Conv1D(32, 8, activation='relu'))
model.add(Dropout(0.7))
model.add(BatchNormalization())
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("./weights.h5",monitor="val_loss",mode="min",save_best_only = True,verbose=1)
model.fit(Xtrain, y=y_train, batch_size=128,epochs=25, verbose=1,validation_data=(Xvalid, y_valid),callbacks=[checkpoint])

In [ ]:
model.load_weights("weights.h5")
pred=model.predict_classes(Xvalid)
print(classification_report(yvalid,pred))
print(confusion_matrix(yvalid, pred))

In [ ]:
!wget "http://www-nlp.stanford.edu/data/glove.840B.300d.zip"

In [ ]:
!unzip 'glove.840B.300d.zip'

In [ ]:
embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]



In [ ]:
xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)
xvalid_glove = np.array(xvalid_glove)

In [ ]:

XGB_glove_clf = xgb.XGBClassifier(nthread=10, silent=False)
XGB_glove_clf.fit(xtrain_glove, ytrain)
predictions = XGB_glove_clf.predict_proba(xvalid_glove)

# print ("logloss: %0.3f " % multiclass_logloss(xvalid_glove, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))

In [ ]:
LR_glove_clf = LogisticRegression(C=1.0)
LR_glove_clf.fit(xtrain_glove, ytrain)
predictions = LR_glove_clf.predict_proba(xvalid_glove)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(yvalid,np.argmax(predictions,axis=1)))
print(confusion_matrix(yvalid, np.argmax(predictions,axis=1)))